In [1]:
import torch
from tvm import relay

## Load Model

In [2]:
from mmdet.apis import init_detector

config = '/home/austin/work/3rd/mmdetection/configs/yolo/yolov3_d53_mstrain-416_273e_coco.py'
checkpoint = '/workspace/datasets/mmcv/models/yolov3_d53_mstrain-416_273e_coco-2b60fcd9.pth'

# initialize the detector
model = init_detector(config, checkpoint, device='cpu')

load checkpoint from local path: /workspace/datasets/mmcv/models/yolov3_d53_mstrain-416_273e_coco-2b60fcd9.pth


搜索resize参数：

In [3]:
test_pipeline = model.cfg.test_pipeline
img_scale = [_dict['img_scale']for _dict in test_pipeline if 'img_scale' in _dict]
assert len(img_scale) == 1 # should only has one 'img_scale'
img_scale = img_scale[0]

## Trace Model

由于mmdetection的`model.forward()`包含了nms等后处理，因此这里替换`forward()`为`forward_dummy()`：

In [4]:
from copy import deepcopy

_model = deepcopy(model)
_model.forward = _model.forward_dummy

In [5]:
input_data = torch.randn((1, 3, *img_scale))
scripted_model = torch.jit.trace(_model, input_data).eval()

/home/austin/work/3rd/mmdetection/mmdet/models/dense_heads/yolo_head.py:126: UserWarning: DeprecationWarning: `anchor_generator` is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: `anchor_generator` is deprecated, '
/home/austin/work/3rd/mmdetection/mmdet/models/dense_heads/yolo_head.py:136: UserWarning: DeprecationWarning: `num_anchors` is deprecated, please use "num_base_priors" instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


In [6]:
shape_list = [('input', input_data.shape)]
mod, params = relay.frontend.from_pytorch(
    scripted_model, shape_list, use_parser_friendly_name=True)

In [7]:
mod['main']

fn (%input: Tensor[(1, 3, 416, 416), float32], %backbone_conv1_conv_weight: Tensor[(32, 3, 3, 3), float32], %backbone_conv1_bn_weight: Tensor[(32), float32], %backbone_conv1_bn_bias: Tensor[(32), float32], %backbone_conv1_bn_running_mean: Tensor[(32), float32], %backbone_conv1_bn_running_var: Tensor[(32), float32], %backbone_conv_res_block1_conv_conv_weight: Tensor[(64, 32, 3, 3), float32], %backbone_conv_res_block1_conv_bn_weight: Tensor[(64), float32], %backbone_conv_res_block1_conv_bn_bias: Tensor[(64), float32], %backbone_conv_res_block1_conv_bn_running_mean: Tensor[(64), float32], %backbone_conv_res_block1_conv_bn_running_var: Tensor[(64), float32], %backbone_conv_res_block1_res0_conv1_conv_weight: Tensor[(32, 64, 1, 1), float32], %backbone_conv_res_block1_res0_conv1_bn_weight: Tensor[(32), float32], %backbone_conv_res_block1_res0_conv1_bn_bias: Tensor[(32), float32], %backbone_conv_res_block1_res0_conv1_bn_running_mean: Tensor[(32), float32], %backbone_conv_res_block1_res0_conv1_